# init

In [2]:
import pygame
import numpy as np
import numpy.linalg as lin
import sys
import importlib
import itertools
import time

import Clipping

from Geometry import *
from camera import Camera
import draw
from inputs import input_update

# reload modules

In [56]:

importlib.reload(Clipping)
importlib.reload(sys.modules['Geometry'])
importlib.reload(sys.modules['vec'])
importlib.reload(sys.modules['util'])
importlib.reload(sys.modules['opengl_draw_2d'])
importlib.reload(sys.modules['opengl_draw_3d'])
from Geometry import *

importlib.reload(sys.modules['camera'])
from camera import Camera
importlib.reload(sys.modules['inputs'])
from inputs import input_update

importlib.reload(sys.modules['draw'])
import draw



# build shapes

In [48]:
from colors import *
    
d = 4 #set dimensionality
cube_face_colors = [RED,YELLOW,GREEN,CYAN,MAGENTA, WHITE]
tess_face_colors = cube_face_colors + [ORANGE,PURPLE]

cube_shape = build_cube(d)


if d == 4:
    for face, color in zip(cube_shape.faces,tess_face_colors):
        face.color = color
    shapes = []
    for i in range(4):
        cube = cube_shape.copy()
        cube.update(pos = 2*np.eye(d)[i])
        shapes.append(cube)
        cube = cube_shape.copy()
        cube.update(pos = -2*np.eye(d)[i])
        shapes.append(cube)
    shapes = [shapes[0]]
    #shapes = [cube_shape] #single cube
    #cube_shape.transparent = True
if d == 3:
    for face, color in zip(cube_shape.faces,cube_face_colors):
        face.color = color
    cube1 = cube_shape.copy()
    cube1.update(pos=np.array([2,0,0]))

    cube2 = cube_shape.copy()
    cube2.update(pos=np.array([-2,0,0]))

    cube3 = cube_shape.copy()
    cube3.update(pos=np.array([0,0,2]))

    cube4 = cube_shape.copy()
    cube4.update(pos=np.array([0,0,-2]))

    cube5 = cube_shape.copy()
    cube5.update(pos=np.array([0,-2,0]))
    
    shapes = [cube1,cube2,cube3,cube4,cube5]

    shapes[0].transparent = True
for i,shape in enumerate(shapes):
    shape.name = 'shape ' + str(i)

shape_combinations = list(itertools.combinations(shapes,2))

# main loop

Would like draw.py to have clearer distinctions between the various spaces. The layers are
1. The d-dim local spaces that shape.verts_ref, ref_frame, and face.normal_ref etc live in
2. The d-dim world space that rotated, scaled and positioned shapes (and faces) and cameras live
3. The d-dim space rotated and translated wrt to a camera
4. The (d-1)-dim space that shapes are projected on to
5. The (d-1)-dim graphics backend (openGL)

It would be nice if we could organize code according to these layers.

Clipping happens in 2

In [101]:
import colors
import vec
def test_points(camera):
    points = [ vec.Vec([np.cos(th),np.sin(th),0]) for th in np.linspace(0,2*np.pi,200)]
    draw.draw_points(camera,points,colors.GREEN)

In [57]:

t = 0
#size = width, height = 1100, 700
size = width, height = 1600, 900

pygame.init()

if d == 3:
    draw.init(d,size,focal=6)
    draw.view_radius = 7

    camera = Camera(pos = Point([0.8,2,0.8]))
    camera.update_rot_matrix(0,-1,-3*np.pi/4)
    camera.update_rot_matrix(1,-1,-np.pi/4)

if d == 4:
    draw.init(d,size,stereo=False,n_fuzz_points = 600)
    if draw.stereo:
        draw.view_radius = 4
    else:
        draw.view_radius = 6
    #draw.face_scales = [0.95,0.5]
    #draw_class.stereo_view_angles = [[-30,0],[0,0]]
    #camera = Camera(pos = Point([0,0,0,0]))
    camera.look_at(shapes[0].pos)
    #camera = Camera(pos = np.array([0.8,2,0.8,0.8]),draw_class = draw_class)
    #camera.update_rot_matrix(0,-1,-3*np.pi/4)
    #camera.update_rot_matrix(2,-1,-np.pi/4)
    #camera.update_rot_matrix(1,-1,-np.pi/4)
#camera.update_rot_matrix(2,3,-np.pi/4)
draw.clipping = True
quit = False
redraw = True
timer = time.time()
frame = 0
try:
    while not quit:

        if redraw:
            draw.draw(camera,shapes)
            #test_points(camera)
            pygame.display.flip()
            redraw = False
            
        #this is here so that we draw on the first frame
        redraw, quit = input_update(camera, draw, shapes)
        shapes[0].update(angles=[0,time.time()])
        redraw = True #redraw every loop. useful for assessing framerate


        delta_t = (time.time() - timer)
        if delta_t > 2:
        #if False:
            #print('redraw:',redraw)
            try:
                #fps = 30/(time.time() - timer)
                fps = frame/delta_t
            except:
                fps = 0
            #if fps < 60:
            print(fps,'fps')
            timer = time.time()
            frame = 0
        frame = frame+1
except:
    pygame.quit()
    print('game terminated')
    raise
pygame.quit()
print('game terminated')

init GL
10.122736960999362 fps
10.70512007525107 fps
10.216981621228864 fps
10.398451383614884 fps
10.74696999060809 fps
10.231948003158323 fps
10.083105582980231 fps
10.848885767753934 fps
10.442284047619358 fps
10.092030252584955 fps
10.57149240075824 fps
10.612350874545776 fps
10.238662079592464 fps
10.617544235122875 fps
10.720594625266461 fps
10.310791997183976 fps
10.334251264585129 fps
10.929905400179214 fps
10.134810806183404 fps
10.15981030489564 fps
10.788628682614833 fps
10.491129550851936 fps
10.274265468458621 fps
10.6480053515423 fps
10.562012932457373 fps
10.115993927673715 fps
10.643991398956834 fps
10.741556746983477 fps
10.164150546476495 fps
10.572137971363988 fps
10.848112856964665 fps
10.153837108861962 fps
10.437981382836895 fps
10.768996854777063 fps
10.416039495633159 fps
9.896023780823443 fps
10.758000995184037 fps
10.643572736766803 fps
10.062158691477258 fps
10.479307740519765 fps
10.745678423466522 fps
12.996859073293493 fps
12.902782397102255 fps
13.0042944

In [24]:
pygame.quit()

In [44]:
[v + vec.Vec([1,1,1,1]) for v in shapes[0].verts_ref]

[array([0., 0., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([0., 0., 1., 0.]),
 array([0., 0., 1., 1.]),
 array([0., 1., 0., 0.]),
 array([0., 1., 0., 1.]),
 array([0., 1., 1., 0.]),
 array([0., 1., 1., 1.]),
 array([1., 0., 0., 0.]),
 array([1., 0., 0., 1.]),
 array([1., 0., 1., 0.]),
 array([1., 0., 1., 1.]),
 array([1., 1., 0., 0.]),
 array([1., 1., 0., 1.]),
 array([1., 1., 1., 0.]),
 array([1., 1., 1., 1.]),
 array([0., 0., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([0., 0., 1., 0.]),
 array([0., 0., 1., 1.]),
 array([0., 1., 0., 0.]),
 array([0., 1., 0., 1.]),
 array([0., 1., 1., 0.]),
 array([0., 1., 1., 1.]),
 array([1., 0., 0., 0.]),
 array([1., 0., 0., 1.]),
 array([1., 0., 1., 0.]),
 array([1., 0., 1., 1.]),
 array([1., 1., 0., 0.]),
 array([1., 1., 0., 1.]),
 array([1., 1., 1., 0.]),
 array([1., 1., 1., 1.])]

In [51]:
shapes[0].verts_ref

array([[-0.5, -0.5, -0.5, -0.5],
       [-0.5, -0.5, -0.5,  0.5],
       [-0.5, -0.5,  0.5, -0.5],
       [-0.5, -0.5,  0.5,  0.5],
       [-0.5,  0.5, -0.5, -0.5],
       [-0.5,  0.5, -0.5,  0.5],
       [-0.5,  0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5,  0.5],
       [ 0.5, -0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0.5, -0.5,  0.5,  0.5],
       [ 0.5,  0.5, -0.5, -0.5],
       [ 0.5,  0.5, -0.5,  0.5],
       [ 0.5,  0.5,  0.5, -0.5],
       [ 0.5,  0.5,  0.5,  0.5],
       [-0.5, -0.5, -0.5, -0.5],
       [-0.5, -0.5, -0.5,  0.5],
       [-0.5, -0.5,  0.5, -0.5],
       [-0.5, -0.5,  0.5,  0.5],
       [-0.5,  0.5, -0.5, -0.5],
       [-0.5,  0.5, -0.5,  0.5],
       [-0.5,  0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5,  0.5],
       [ 0.5, -0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0.5, -0.5,  0.5,  0.5],
       [ 0.5,  0.5, -0.5, -0.5],
       [ 0.5,  0.5, -0.5,  0.5],
       [ 0